In [1]:
// ----------
// elasticsearch + nodejs 
// ----------

// install
`
npm i --save @elastic/elasticsearch
`

// docs
  // https://www.npmjs.com/package/@elastic/elasticsearch

'\nnpm i --save @elastic/elasticsearch\n'

In [2]:
// ----------
// create client
// ----------

// import
const { Client } = require('@elastic/elasticsearch');

// instantiate
const client = new Client({ nodes: [
    'http://localhost:9200',
    'http://localhost:9201',
    'http://localhost:9202'
]});

// const client = new Client({ node: 'http://localhost:9200'});

In [3]:
// ------------
// index data
// ------------

const indexDocument = async (client, config) => {
  try {
    const result = await client.index(config);
    return { data: result, error: null }
  } catch (err) {
    return { data: null, error: err.message }
  }
}

(async (client) => {
  await indexDocument(client, {
    index: 'jonin',
    body: {
      first_name: "Kakashi",
      last_name: "Hatake",
      is_awesome: 'yes'        
    }
  });
  await indexDocument(client, {
    index: 'jonin',
    body: {
      first_name: "Tenzo",
      last_name: "Yamato",
      is_awesome: 'yes'        
    }
  });
})(client)

In [4]:
// ------------
// refresh index
// ------------

const refreshIndex = async (client, config) => {
  try {
    await client.indices.refresh(config)
    return { data: 'success', error: null }
  } catch (err) {
    return { data: null, error: err.message }
  }
};

(async (client) => {
  await refreshIndex(client, {
    index: 'jonin'
  });
})(client)

In [5]:
// ------------
// search
// ------------

const searchDocuments = async (client, config) => {
  try {
    const result = await client.search(config);
    return { data: result, error: null }
  } catch (err) {
    return { data: null, error: err.message }
  }
}

/*
// output structure
{
  body: object | boolean
  statusCode: number
  headers: object
  warnings: [string]
  meta: object
}
*/

// example

(async (client) => {
  const esResponse = await searchDocuments(client, {
    index: 'jonin',
    body: {
      query: {
        match: { firstName: "Kakashi" }
      }
    }
  });
  const { body } = esResponse.data;
  console.log(body);
})(client)


{
  took: 1,
  timed_out: false,
  _shards: { total: 1, successful: 1, skipped: 0, failed: 0 },
  hits: { total: { value: 0, relation: 'eq' }, max_score: null, hits: [] }
}


In [6]:
// ----------
// count
// ----------

const countDocuments = async (client, config) => {
  try {
    const count = await client.count(config);
    return { data: count, error: null };
  } catch (err) {
    return { data: null, error: err.message }
  }
}

// example

(async (client) => {
  const esResponse = await countDocuments(client, {
    index: 'jonin'
  })
  console.log(esResponse.data);
})(client)


{
  body: {
    count: 2,
    _shards: { total: 1, successful: 1, skipped: 0, failed: 0 }
  },
  statusCode: 200,
  headers: {
    'content-type': 'application/json; charset=UTF-8',
    'content-length': '71'
  },
  warnings: null,
  meta: {
    context: null,
    request: { params: [Object], options: [Object], id: 5 },
    name: 'elasticsearch-js',
    connection: {
      url: 'http://localhost:9200/',
      id: 'http://localhost:9200/',
      headers: {},
      deadCount: 0,
      resurrectTimeout: 0,
      _openRequests: 0,
      status: 'alive',
      roles: [Object]
    },
    attempts: 0,
    aborted: false
  }
}


In [7]:
// ----------
// update
// ----------

const updateDocument = async (client, config) => {
  try {
    await client.update(config);
    return { data: 'success', error: null };
  } catch (err) {
    return { data: null, error: err.message }
  }
}

// example

(async (client) => {
  // update
  await updateDocument(client, {
    index: 'jonin',
    id: 1,
    body: {
      doc: {
        last_name: "HATAKE"
      }
    }
  })
  await updateDocument(client, {
    index: 'jonin',
    id: 2,
    body: {
      doc: {
        last_name: "YAMATO"
      }
    }
  });
  // refresh
  await refreshIndex(client, { index: 'jonin' });
  // search
  const esResponse = await searchDocuments(client, {
    index: 'jonin',
    body: {
      query: {
        match_all: {}
      }
    }
  });
  const { body } = esResponse.data;
  console.log(body);

})(client)


{
  took: 2,
  timed_out: false,
  _shards: { total: 1, successful: 1, skipped: 0, failed: 0 },
  hits: {
    total: { value: 2, relation: 'eq' },
    max_score: 1,
    hits: [ [Object], [Object] ]
  }
}


In [8]:
// ----------
// update by query
// ----------

// *** NOT WORKING ***

const updateDocumentsByQuery = async (client, config) => {
  try {
    await client.update(config);
    return { data: 'success', error: null };
  } catch (err) {
    return { data: null, error: err.message }
  }
}

(async (client) => {
  // update
  await updateDocumentsByQuery(client, {
    index: 'jonin',
    body: {
      query: {
        match_all: {}
      },
      script: {
        source: "ctx._source.is_awesome = params.new_value",
        params: {
          new_value: 'definitely'
        }
      }
    }
  })
  // refresh
  await refreshIndex(client, { index: 'jonin' });
  // search
  const esResponse = await searchDocuments(client, {
    index: 'jonin',
    body: {
      query: {
        match_all: {}
      }
    }
  });
  const { body } = esResponse.data;
  console.log(body);
  console.log(JSON.stringify(body.hits.hits, null, 2))
})(client)


{
  took: 4,
  timed_out: false,
  _shards: { total: 1, successful: 1, skipped: 0, failed: 0 },
  hits: {
    total: { value: 2, relation: 'eq' },
    max_score: 1,
    hits: [ [Object], [Object] ]
  }
}
[
  {
    "_index": "jonin",
    "_type": "_doc",
    "_id": "6AGMyXEBAB4tac_CjboG",
    "_score": 1,
    "_source": {
      "first_name": "Kakashi",
      "last_name": "Hatake",
      "is_awesome": "yes"
    }
  },
  {
    "_index": "jonin",
    "_type": "_doc",
    "_id": "6QGMyXEBAB4tac_Cjbo0",
    "_score": 1,
    "_source": {
      "first_name": "Tenzo",
      "last_name": "Yamato",
      "is_awesome": "yes"
    }
  }
]


In [9]:
// ----------
// delete
// ----------

const deleteDocumentsByQuery = async (client, config) => {
  try {
    await client.deleteByQuery(config);
    return { data: 'success', error: null };
  } catch (err) {
    return { data: null, error: err.message }
  }
}

// example

(async (client) => {
  await deleteDocumentsByQuery(client, {
    index: 'jonin',
    body: {
      query: {
        match_all: {} // delete all
      }
    }
  })
})(client)